# Nettoyage de la données

In [1]:
from data_prep import DataProcessor


In [2]:
import pandas as pd

In [5]:
data_file = '/Users/camille/repo/Hetic/projet_gouv/scraping/all_data/google_reviews_RGPD.csv'
df_scraped_data = pd.read_csv(data_file)
columns = df_scraped_data.columns
# Préparation des données
data_processor = DataProcessor(data_file)
data_processor.process_data(columns)
data = data_processor.data

In [6]:
data

,date,rate,review_text,object_address,review_num,years
0,2023-04-02,1,Pas de formation possible\nPas de financement ...,"5 Rue de Quimper, 68100 Mulhouse",32,2023
1,2023-05-02,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32,2023
2,2023-02-02,1,"Les fonctionnaires incompétents, arrogants et ...","5 Rue de Quimper, 68100 Mulhouse",32,2023
3,2022-09-02,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32,2022
4,2022-09-02,1,Si je pouvais ne pas mettre d'étoiles je n'en ...,"5 Rue de Quimper, 68100 Mulhouse",32,2022
...,...,...,...,...,...,...
16640,2020-06-02,5,NaN,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2020
16641,2018-06-02,1,NaN,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2018
16642,2022-06-02,1,NaN,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2022
16643,2022-11-02,1,NaN,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2022


In [59]:
import pandas as pd
import spacy

# Charger le modèle de langue français
nlp = spacy.load("fr_core_news_sm")
nlp.max_length = 2000000



dependances_incluses = ['nsubj', 'xcomp', 'obl:mod', 'acl', 'conj', 'amod']
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

data['cleaned_text'] = data['review_text'].astype(str)
data.dropna(inplace=True)



# Lemmatisation des termes des avis négatifs
data_terms = data['cleaned_text'].apply(lambda x: [token.lemma_ for token in nlp(x) if token.dep_ in dependances_incluses and token.text.lower() not in stop_words])
#delete ponctuation sur cleaned_text
data['cleaned_text'] = data['cleaned_text'].str.replace('[^\w\s]','')
#  lower sur cleaned_text
data['cleaned_text'] = data['cleaned_text'].str.lower()
# delete \n sur cleaned_text
data['cleaned_text'] = data['cleaned_text'].str.replace('\n','')
# delete stop_words sur cleaned_text
data['cleaned_text'] = data['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# Ajouter les colonnes pos_reviews_terms et data_terms au DataFrame
data['data_terms'] = data_terms

# Enregistrement des données avec les colonnes ajoutées


In [73]:
# utilse que years ==2023 
data_2023 = data[data['years'] == '2023']
# groupe par mois sans de colonne mois 
data_2023['date'] = pd.to_datetime(data_2023['date'])
data['sentiment'] = data['rate'].apply(lambda x: 1 if x > 2 else 0)


In [74]:
data

,date,rate,review_text,object_address,review_num,years,cleaned_text,data_terms,sentiment
0,2023-04-02,1,Pas de formation possible\nPas de financement ...,"5 Rue de Quimper, 68100 Mulhouse",32,2023,formation possiblepas financement possiblerefu...,"[rendre, inutile, réactif]",0
1,2023-05-02,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32,2023,aucun respect jexplique pourtant bien soucis e...,"[respect, souci, comprendre, tourner, rond, do...",0
2,2023-02-02,1,"Les fonctionnaires incompétents, arrogants et ...","5 Rue de Quimper, 68100 Mulhouse",32,2023,fonctionnaires incompétents arrogants prétenti...,"[fonctionnaire, arrogant, prétentieux, transfo...",0
3,2022-09-02,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32,2022,aucun respect jexplique pourtant bien soucis e...,"[respect, souci, comprendre, hautain, discrimi...",0
4,2022-09-02,1,Si je pouvais ne pas mettre d'étoiles je n'en ...,"5 Rue de Quimper, 68100 Mulhouse",32,2022,pouvais mettre détoiles nen mettrais conseillè...,"[mettre, hyper, désagréable, agacer, échange, ...",0
...,...,...,...,...,...,...,...,...,...
16623,2015-06-02,3,j'ai eu soixante ans le 15 mars ; suis je obli...,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2015,jai ans 15 mars obligé pointé pole emploi,"[an, 15]",1
16624,2017-06-02,3,Un pôle emploi comme un autre,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2017,pôle emploi,[emploi],1
16625,2022-06-02,1,Incompétents.,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2022,incompétents,[],0
16626,2020-06-02,1,Sourires en option,"45 Av. Billaud Varenne, 17000 La Rochelle",34,2020,sourires option,[],0
